In [1]:
!pip install BeautifulSoup4
!pip install requests
!pip install pymysql
!pip install pandas

In [2]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

In [4]:
import pymysql as pms
import datetime
import re

### 1. 판례 목록 나와있는 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* search	int	검색범위 (기본 : 1 판례명) 2 : 본문검색
* query	string	검색범위에서 검색을 원하는 질의(검색 결과 리스트)
* display	int	검색된 결과 개수 (default=20 max=100)
* page	int	검색 결과 페이지 (default=1)
* org	string	법원종류 (대법원:400201, 하위법원:400202)
* curt	string	법원명 (대법원, 서울고등법원, 광주지법, 인천지방법원)
* JO	string	참조법령명(형법, 민법 등)
* gana	string	사전식 검색(ga,na,da…,etc)
* sort	string	정렬옵션
* lasc : 사건명 오름차순
* ldes : 사건명 내림차순
* dasc : 선고일자 오름차순
* ddes : 선고일자 내림차순(생략시 기본)
* nasc : 법원명 오름차순
* ndes : 법원명 내림차순
* date	int	판례 선고일자
* prncYd	string	선고일자 검색(20090101~20090130)
* nb	int	판례 사건번호
* popYn	string	상세화면 팝업창 여부(팝업창으로 띄우고 싶을 때만 'popYn=Y')
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 목록 parameter 딕셔너리


> query가 keyword에 해당하는데, input 함수로 키워드를 입력받음
<br/> 여기선 키워드로 손해배상 입력, 보여줄 판례의 수는 10개, 법원은 대법원, 기간은 20100101~20191231로 설정함
<br/> type은 url 타고 웹에서 볼거면 html로 하는게 낫고, 주피터노트북으로 text 확인할거면 xml이 나음

In [13]:
chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':input()
        ,'display':1000
        ,'curt':'대법원'
        ,'prncYd':'20100101~20191231'
       }

손해배상


* #### request 모듈을 이용하여 chart의 변수들을 전송함

In [5]:
res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)

* #### url 가져오기

In [6]:
print(res_chart.url)

http://www.law.go.kr/DRF/lawSearch.do?OC=ICTPoolC&target=prec&type=XML&query=%EC%86%90%ED%95%B4%EB%B0%B0%EC%83%81&display=1000&curt=%EB%8C%80%EB%B2%95%EC%9B%90&prncYd=20100101~20191231


* #### text 가져오기
> 아래에서 볼 수 있듯이 html 타입으로 text 읽어오는건 무의미한듯..

In [7]:
print(res_chart.text)

<?xml version="1.0" encoding="UTF-8"?><PrecSearch><target>prec</target><키워드>손해배상</키워드><section>evtNm</section><totalCnt>880</totalCnt><page>1</page><prec id="1"><판례일련번호>210813</판례일련번호><사건명><![CDATA[<strong class="tbl_tx_type"><strong class="tbl_tx_type">손해</strong><strong class="tbl_tx_type">배상</strong></strong>(기)]]></사건명><사건번호>2015다225776</사건번호><선고일자>2019.11.28</선고일자><법원명>대법원</법원명><법원종류코드></법원종류코드><사건종류명>민사</사건종류명><사건종류코드>400101</사건종류코드><판결유형>판결</판결유형><선고>선고</선고><판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210813&amp;type=HTML&amp;mobileYn=</판례상세링크></prec><prec id="2"><판례일련번호>210827</판례일련번호><사건명><![CDATA[<strong class="tbl_tx_type"><strong class="tbl_tx_type">손해</strong><strong class="tbl_tx_type">배상</strong></strong>(기)]]></사건명><사건번호>2019다233072</사건번호><선고일자>2019.11.14</선고일자><법원명>대법원</법원명><법원종류코드></법원종류코드><사건종류명>민사</사건종류명><사건종류코드>400101</사건종류코드><판결유형>판결</판결유형><선고>선고</선고><판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210827&amp;type=HTML&amp;mobileYn=</

### 2. 판례 본문 url 가져오기
> 입력 변수
* OC	string(필수)	사용자 이메일의 ID(g4c@korea.kr일경우 OC값=g4c)
* target	string : prec(필수)	서비스 대상
* type	char(필수)	출력 형태 : HTML / XML
* ID	char(필수)	판례 일련번호
* LM	string	판례명
> ##### (필수) 제외하면 안들어가도 되는 변수임
</br>

* #### 본문 parameter 딕셔너리


> ID가 판례일련번호에 해당하는데, input 함수로 판례일련번호를 입력받음

In [9]:
main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': int(input()),
          'type':'XML'}

KeyboardInterrupt: Interrupted by user

* #### request 모듈을 이용하여 main의 변수들을 전송함

In [ ]:
res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)

* #### url 가져오기

In [ ]:
print(res_main.url)

### 3. 판례 목록에서 판례일련번호만 리스트에 저장하기
> tag로부터 판례일련번호를 가져와야하므로 beautifulSoup 모듈이용

In [ ]:
!pip uninstall lxml

In [ ]:
soup = bs(res_chart.text,'xml')

In [ ]:
find = soup.find_all('판례일련번호')

In [ ]:
find

#### 번호만 가져와야 하므로 slicing 이용

In [ ]:
num_list = []
for k in find:
    line = str(k)
    num_list.append(int(line[8:14]))

In [ ]:
num_list

### 4. 함수
> 두가지의 함수를 만들었다
> * 키워드와 열람하고싶은 판례의 수를 입력하면, 해당 판례들의 일련번호를 리스트로 반환하는 함수
> * 위의 함수를 이용해 일련번호 리스트 안의 각 번호에 해당하는 판례의 url을 제공하는 함수

In [ ]:
def get_numbers(keyword,num):
    chart= {'OC':'ICTPoolC'
        ,'target':'prec'
        ,'type':'XML'
        ,'query':keyword
        ,'display':num
        ,'curt':'대법원'
        ,'prncYd':'20100101~20191231'
       }
    res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)
    soup = bs(res_chart.text,'xml')
    find = soup.find_all('판례일련번호')
    num_list = []
    for k in find:
        line = str(k)
        num_list.append(int(line[8:14]))
    return num_list

In [ ]:
get_numbers('손해배상',1000)

In [ ]:
def get_url(keyword,num):
    num_list = get_numbers(keyword,num)
    url_list = []
    for i in num_list:
        main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': i,
          'type':'HTML'}
        res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)
        url_list.append(res_main.url)
    for url in url_list:
        print(url)

In [ ]:
get_url('손해배상', 10)

### 4. DB연동

In [10]:
!pip install cryptography

     |████████████████████████████████| 1.8 MB 389 kB/s eta 0:00:01
     |████████████████████████████████| 175 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 5.4 MB/s eta 0:00:01


In [126]:
# 1. DB 연결
conn = pms.connect(host="ict.clw3yf0azeqw.ap-northeast-2.rds.amazonaws.com", port=3306, user="admin", password="48834883", db="ict", charset="utf8")

In [9]:
soup = bs(res_chart.text,'xml')

In [10]:
ids = soup.find_all('판례일련번호')
ids

[<판례일련번호>210813</판례일련번호>,
 <판례일련번호>210827</판례일련번호>,
 <판례일련번호>210829</판례일련번호>,
 <판례일련번호>210569</판례일련번호>,
 <판례일련번호>210415</판례일련번호>,
 <판례일련번호>210423</판례일련번호>,
 <판례일련번호>210419</판례일련번호>,
 <판례일련번호>210853</판례일련번호>,
 <판례일련번호>210895</판례일련번호>,
 <판례일련번호>209687</판례일련번호>,
 <판례일련번호>209693</판례일련번호>,
 <판례일련번호>209587</판례일련번호>,
 <판례일련번호>209131</판례일련번호>,
 <판례일련번호>208800</판례일련번호>,
 <판례일련번호>208795</판례일련번호>,
 <판례일련번호>208726</판례일련번호>,
 <판례일련번호>208733</판례일련번호>,
 <판례일련번호>208734</판례일련번호>,
 <판례일련번호>208744</판례일련번호>,
 <판례일련번호>208745</판례일련번호>,
 <판례일련번호>208415</판례일련번호>,
 <판례일련번호>207529</판례일련번호>,
 <판례일련번호>207541</판례일련번호>,
 <판례일련번호>207485</판례일련번호>,
 <판례일련번호>207486</판례일련번호>,
 <판례일련번호>207490</판례일련번호>,
 <판례일련번호>207140</판례일련번호>,
 <판례일련번호>207095</판례일련번호>,
 <판례일련번호>207100</판례일련번호>,
 <판례일련번호>207107</판례일련번호>,
 <판례일련번호>206286</판례일련번호>,
 <판례일련번호>205697</판례일련번호>,
 <판례일련번호>205700</판례일련번호>,
 <판례일련번호>205791</판례일련번호>,
 <판례일련번호>205368</판례일련번호>,
 <판례일련번호>204520</판례일련번호>,
 <판례일련번호>204471</판례일련번호>,
 <판례일련번호>204473</판례일련번호>,
 <판례일련번호>204

In [11]:
id_list = []
for id in ids:
    line = str(id)
    id_list.append(int(line[8:-9]))
id_list

[210813,
 210827,
 210829,
 210569,
 210415,
 210423,
 210419,
 210853,
 210895,
 209687,
 209693,
 209587,
 209131,
 208800,
 208795,
 208726,
 208733,
 208734,
 208744,
 208745,
 208415,
 207529,
 207541,
 207485,
 207486,
 207490,
 207140,
 207095,
 207100,
 207107,
 206286,
 205697,
 205700,
 205791,
 205368,
 204520,
 204471,
 204473,
 204489,
 204501,
 204506,
 208848,
 204343,
 208849,
 204331,
 204335,
 204347,
 207581,
 204219,
 207580,
 207579,
 204226,
 204230,
 207594,
 207588,
 204247,
 209581,
 206325,
 206320,
 206327,
 204270,
 206331,
 206330,
 204274,
 204295,
 204297,
 206338,
 204201,
 204189,
 203859,
 206347,
 206345,
 206346,
 206352,
 203858,
 203880,
 206342,
 206344,
 206351,
 203771,
 203769,
 205718,
 203976,
 203776,
 205721,
 205720,
 204194,
 203033,
 203020,
 205730,
 204566,
 204578,
 201270,
 201274,
 199836,
 204775,
 204882,
 204884,
 204885,
 204886,
 199864,
 199870,
 204889,
 197597,
 204895,
 197037,
 204900,
 196004,
 195385,
 194926,
 194897,
 

In [12]:
caseNames = soup.find_all('사건명')
caseNames

[<사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)</사건명>,
 <사건명>&lt;strong class="tbl_tx_type"&gt;&lt;strong class="tbl_tx_type"&gt;손해&lt;/strong&gt;&lt;strong class="tbl_tx_type"&gt;배상&lt;/strong&gt;&lt;/strong&gt;(기)

In [91]:
caseName_list = []
for title in caseNames:
    line = title.string
    line = re.sub('<.*?>','',line);
    caseName_list.append(line)
    #print(line)
caseName_list

['손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '임대차보증금·손해배상(기)',
 '손해배상',
 '손해배상(기)·손해배상(기)·손해배상(기)',
 '손해배상(자)(교통사고 피해자의 유족들이 가해자와 보험사를 상대로 손해배상을 구하는 사건)',
 '영업비밀침해금지및손해배상등',
 '손해배상',
 '손해배상(기)',
 '손해배상(자)(교통사고로 발생한 장애로 인한 손해배상청구권의 소멸시효의 기산일)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)(상가임대차법상 권리금 회수 방해를 이유로 한 손해배상청구 사건)',
 '손해배상(기)등',
 "손해배상(기)('종북의 상징'이라는 표현행위로 인한 인격권 침해를 이유로 한 위자료 청구 사건)",
 '손해배상(기)',
 '손해배상(자)(원고가 교통사고로 말미암은 후유장해로 복합부위통증증후군(CRPS)을 주장하면서 손해배상을 청구하는 사건)',
 '손해배상(기)·건물인도등(임차인이 임대인을 상대로 권리금 회수 방해로 인한 손해배상을 구하는 사건)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(지)',
 '손해배상(자)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(지)',
 '손해배상(기)',
 '손해배상(기)(도시일용노동의 경험칙상 가동연한)',
 '손해배상(의)',
 '손해배상(기)',
 '특허침해금지및손해배상청구의소',
 '손해배상(기)',
 '손해배상(지)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상',
 '손해배상및매매대금반환',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상',
 '손해배상(기)',
 '손해배상(기)',
 '손해배상청구등·부당이득금반환',
 '손해배상(기)',
 '임금·손해배상(기)',
 '손해배상(의)'

In [92]:
caseNums = soup.find_all('사건번호')
caseNums

[<사건번호>2015다225776</사건번호>,
 <사건번호>2019다233072</사건번호>,
 <사건번호>2019다216312</사건번호>,
 <사건번호>2018다233686</사건번호>,
 <사건번호>2018다233242</사건번호>,
 <사건번호>2017다282438</사건번호>,
 <사건번호>2016다243306</사건번호>,
 <사건번호>2017다204490</사건번호>,
 <사건번호>2019다14998</사건번호>,
 <사건번호>2018다222303</사건번호>,
 <사건번호>2017다280951</사건번호>,
 <사건번호>2017다34981</사건번호>,
 <사건번호>2016다217833</사건번호>,
 <사건번호>2019다206933</사건번호>,
 <사건번호>2016다1687</사건번호>,
 <사건번호>2016다224626</사건번호>,
 <사건번호>2019다213009</사건번호>,
 <사건번호>2018다239608</사건번호>,
 <사건번호>2018다284226</사건번호>,
 <사건번호>2017다17436</사건번호>,
 <사건번호>2014다220798</사건번호>,
 <사건번호>2017다53265</사건번호>,
 <사건번호>2015다8902</사건번호>,
 <사건번호>2017다225312</사건번호>,
 <사건번호>2016다260455</사건번호>,
 <사건번호>2017다239311</사건번호>,
 <사건번호>2018다287362</사건번호>,
 <사건번호>2018다300708</사건번호>,
 <사건번호>2016다276719</사건번호>,
 <사건번호>2017마6337</사건번호>,
 <사건번호>2018다286550</사건번호>,
 <사건번호>2017다290095</사건번호>,
 <사건번호>2016다271608</사건번호>,
 <사건번호>2018다248909</사건번호>,
 <사건번호>2017다203763</사건번호>,
 <사건번호>2018다255105</사건번호>,
 <사건번호>2018다267252</사건번호>,
 <사건번호>2016

In [93]:
caseNum_list = []
for num in caseNums:
    line = str(num)
    caseNum_list.append(line[6:-7])
caseNum_list

['2015다225776',
 '2019다233072',
 '2019다216312',
 '2018다233686',
 '2018다233242',
 '2017다282438',
 '2016다243306',
 '2017다204490',
 '2019다14998',
 '2018다222303',
 '2017다280951',
 '2017다34981',
 '2016다217833',
 '2019다206933',
 '2016다1687',
 '2016다224626',
 '2019다213009',
 '2018다239608',
 '2018다284226',
 '2017다17436',
 '2014다220798',
 '2017다53265',
 '2015다8902',
 '2017다225312',
 '2016다260455',
 '2017다239311',
 '2018다287362',
 '2018다300708',
 '2016다276719',
 '2017마6337',
 '2018다286550',
 '2017다290095',
 '2016다271608',
 '2018다248909',
 '2017다203763',
 '2018다255105',
 '2018다267252',
 '2016다258148',
 '2017다289903',
 '2017다203596',
 '2017다284885',
 '2016다236131',
 '2016두60287',
 '2017다1448',
 '2018다24349',
 '2015다236196',
 '2018다245702',
 '2018다214142',
 '2016다33196',
 '2017다256910',
 '2017다207994',
 '2016다266736',
 '2016다274270',
 '2016다43872',
 '2018다245160',
 '2018다10562',
 '2016다259905',
 '2013다67587',
 '2017다207529',
 '2015다45420',
 '2016다266606',
 '2016다12748',
 '2016다11226',
 '2017다35717'

In [94]:
dates = soup.find_all('선고일자')
dates

[<선고일자>2019.11.28</선고일자>,
 <선고일자>2019.11.14</선고일자>,
 <선고일자>2019.11.14</선고일자>,
 <선고일자>2019.11.14</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.31</선고일자>,
 <선고일자>2019.10.17</선고일자>,
 <선고일자>2019.09.26</선고일자>,
 <선고일자>2019.09.26</선고일자>,
 <선고일자>2019.09.10</선고일자>,
 <선고일자>2019.08.14</선고일자>,
 <선고일자>2019.07.25</선고일자>,
 <선고일자>2019.07.25</선고일자>,
 <선고일자>2019.07.11</선고일자>,
 <선고일자>2019.07.10</선고일자>,
 <선고일자>2019.07.10</선고일자>,
 <선고일자>2019.07.04</선고일자>,
 <선고일자>2019.07.04</선고일자>,
 <선고일자>2019.06.13</선고일자>,
 <선고일자>2019.05.30</선고일자>,
 <선고일자>2019.05.30</선고일자>,
 <선고일자>2019.05.16</선고일자>,
 <선고일자>2019.05.16</선고일자>,
 <선고일자>2019.05.10</선고일자>,
 <선고일자>2019.04.25</선고일자>,
 <선고일자>2019.04.11</선고일자>,
 <선고일자>2019.04.11</선고일자>,
 <선고일자>2019.04.10</선고일자>,
 <선고일자>2019.04.03</선고일자>,
 <선고일자>2019.02.28</선고일자>,
 <선고일자>2019.02.28</선고일자>,
 <선고일자>2019.02.21</선고일자>,
 <선고일자>2019.02.14</선고일자>,
 <선고일자>2019.01.31</선고일자>,
 <선고일자>2019.01.31</선고일자>,
 <선고일자>2019.01.31</선고일자>,
 <선고일자>2019.

In [95]:
date_list = []
for date in dates:
    line = str(date)
    date_list.append(datetime.date(int(line[6:10]), int(line[11:13]), int(line[14:16])))
date_list

[datetime.date(2019, 11, 28),
 datetime.date(2019, 11, 14),
 datetime.date(2019, 11, 14),
 datetime.date(2019, 11, 14),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 31),
 datetime.date(2019, 10, 17),
 datetime.date(2019, 9, 26),
 datetime.date(2019, 9, 26),
 datetime.date(2019, 9, 10),
 datetime.date(2019, 8, 14),
 datetime.date(2019, 7, 25),
 datetime.date(2019, 7, 25),
 datetime.date(2019, 7, 11),
 datetime.date(2019, 7, 10),
 datetime.date(2019, 7, 10),
 datetime.date(2019, 7, 4),
 datetime.date(2019, 7, 4),
 datetime.date(2019, 6, 13),
 datetime.date(2019, 5, 30),
 datetime.date(2019, 5, 30),
 datetime.date(2019, 5, 16),
 datetime.date(2019, 5, 16),
 datetime.date(2019, 5, 10),
 datetime.date(2019, 4, 25),
 datetime.date(2019, 4, 11),
 datetime.date(2019, 4, 11),
 datetime.date(2019, 4, 10),
 datetime.date(2019, 4, 3),
 datetime.date(2019, 2, 28),
 datetime.date(2019, 2, 28),
 datetime.date(2019, 2, 21),
 datetim

In [96]:
courtNames = soup.find_all('법원명')
courtNames

[<법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명>,
 <법원명>대법원</법원명

In [97]:
courtName_list = []
for courtName in courtNames:
    line = str(courtName)
    courtName_list.append(line[5:-6])
courtName_list

['대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',
 '대법원',


In [98]:
courtIDs = soup.find_all('법원종류코드')
courtIDs

[<법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원종류코드/>,
 <법원

In [99]:
courtID_list = []
for courtID in courtIDs:
    line = str(courtID)
    if line[8:-9] == "":
        courtID_list.append("NULL")
    else:
        courtID_list.append(int(line[8:-9]))
courtID_list

['NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 'NULL',
 

In [100]:
caseTypes = soup.find_all('사건종류명')
caseTypes

[<사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>일반행정</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>,
 <사건종류명>민사</사건종류명>

In [101]:
caseType_list = []
for caseType in caseTypes:
    line = str(caseType)
    caseType_list.append(line[7:-8])
caseType_list

['민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '일반행정',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사',
 '민사

In [102]:
caseTypeIDs = soup.find_all('사건종류코드')
caseTypeIDs

[<사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400101</사건종류코드>,
 <사건종류코드>400

In [103]:
caseTypeID_list = []
for caseTypeID in caseTypeIDs:
    line = str(caseTypeID)
    if line[8:-9] == "":
        caseTypeID_list.append("NULL")
    else:
        caseTypeID_list.append(int(line[8:-9]))
caseTypeID_list

[400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400107,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 400101,
 

In [104]:
verdictTypes = soup.find_all('판결유형')
verdictTypes

[<판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>결정</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>전원합의체 판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결유형>판결</판결유형>,
 <판결

In [105]:
verdictType_list = []
for verdictType in verdictTypes:
    line = str(verdictType)
    if line[6:-7] == "":
        verdictType_list.append("NULL")
    else:
        verdictType_list.append(line[6:-7])
verdictType_list

['판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '결정',
 '판결',
 '판결',
 '판결',
 '전원합의체 판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '전원합의체 판결',
 '전원합의체 판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '결정',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '판결',
 '

In [106]:
sentences = soup.find_all('선고')
sentences

[<선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>자</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고</선고>,
 <선고>선고

In [107]:
sentence_list = []
for sentence in sentences:
    line = str(sentence)
    if line[4:-5] == "":
        sentence_list.append("NULL")
    else:
        sentence_list.append(line[4:-5])
sentence_list

['선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '자',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '자',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 '선고',
 

In [108]:
exURLs = soup.find_all('판례상세링크')
exURLs

[<판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210813&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210827&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210829&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210569&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210415&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210423&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210419&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210853&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210895&amp;type=HTML&amp;mobileYn=</판례상세링크>,
 <판례상세링크>/DRF/lawService.do?

In [109]:
exURL_list = []
for exURL in exURLs:
    line = str(exURL)
    exURL_list.append(line[8:-9])
exURL_list

['/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210813&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210827&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210829&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210569&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210415&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210423&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210419&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210853&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=210895&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209687&amp;type=HTML&amp;mobileYn=',
 '/DRF/lawService.do?OC=ICTPoolC&amp;target=prec&amp;ID=209693&amp;typ

In [116]:
# 2. 커서 생성 (트럭, 연결로프)
cur = conn.cursor()

for i in range (0, len(id_list)):
    sql = "INSERT INTO exlist VALUES( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (id_list[i], caseName_list[i], caseNum_list[i], date_list[i], courtName_list[i], int(), caseType_list[i], caseTypeID_list[i], verdictType_list[i], sentence_list[i], exURL_list[i])
    cur.execute(sql, val)

cur.close()
conn.commit()
print('OK~')

OK~


In [127]:
cur = conn.cursor()

sql = "SELECT ID FROM exlist"
cur.execute(sql)
result = cur.fetchall()

cur.close()
conn.commit()

print(result)

((142237,), (142242,), (142243,), (142245,), (142247,), (142594,), (142595,), (142596,), (142600,), (142601,), (142602,), (142603,), (142611,), (142977,), (143118,), (143119,), (143120,), (143375,), (143377,), (143609,), (143615,), (143632,), (143639,), (143641,), (143643,), (143647,), (143818,), (143819,), (143842,), (143847,), (143889,), (143894,), (143970,), (143974,), (144006,), (144221,), (144227,), (144231,), (144234,), (144235,), (144500,), (144501,), (144515,), (144516,), (144517,), (144522,), (144523,), (144534,), (144685,), (144686,), (144704,), (144707,), (144848,), (144853,), (144857,), (144862,), (144863,), (145101,), (145102,), (145107,), (145139,), (145151,), (145152,), (145161,), (145261,), (145477,), (145716,), (145724,), (145733,), (146168,), (146169,), (146175,), (146176,), (146211,), (146234,), (146240,), (146242,), (146243,), (146326,), (146327,), (146571,), (146589,), (146592,), (146596,), (146625,), (146661,), (146664,), (146665,), (146669,), (146672,), (146744,)

In [129]:
main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': result[0],
          'type':'XML'}

In [130]:
res_main= rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)

In [131]:
print(res_main.text)

<?xml version="1.0" encoding="UTF-8"?><PrecService><판례정보일련번호>142237</판례정보일련번호><사건명><![CDATA[손해배상(기)·공사대금]]></사건명><사건번호>2009다41137,41144</사건번호><선고일자>20100128</선고일자><선고>선고</선고><법원명>대법원</법원명><법원종류코드>400201</법원종류코드><사건종류명>민사</사건종류명><사건종류코드>400101</사건종류코드><판결유형>판결</판결유형><판시사항><![CDATA[[1] 공사도급계약을 체결하면서 지체상금약정과 별도로 손해배상약정을 한 경우, 부실공사와 같은 불완전급부 등으로 발생한 손해에 대하여 위 손해배상약정에 기하여 별도로 그 배상을 청구할 수 있는지 여부(원칙적 적극) 및 이 경우 손해배상의 범위가 지체상금약정에 기한 지체상금액을 초과할 수 없는지 여부(소극)<br/>[2] 수급인이 완공기한 내에 공사를 완성하지 못한 채 중단하여 계약이 해제된 결과 완공이 지연된 경우, 지체상금의 발생시기 및 종기<br/>]]></판시사항><판결요지><![CDATA[[1] 공사도급계약을 체결하면서 건설교통부 고시 ‘민간건설공사 표준도급계약 일반조건’을 계약의 일부로 편입하기로 합의하였고, 위 일반조건에서 지체상금에 관한 규정과 별도로 계약의 해제·해지로 인한 손해배상청구에 관한 규정을 두고 있는 경우, 채무불이행에 관한 손해배상액의 예정은 당사자의 합의로 행하여지는 것으로서, 그 내용이 어떠한가, 특히 어떠한 유형의 채무불이행에 관한 손해배상을 예정한 것인가는 무엇보다도 당해 약정의 해석에 의하여 정하여지는바, 위 일반조건의 지체상금약정은 수급인이 공사완성의 기한 내에 공사를 완성하지 못한 경우에 완공의 지체로 인한 손해배상책임에 관하여 손해배상액을 예정하였다고 해석할 것이고, 수급인이 완공의 지체가 아니라 그 공사를 부실하게 한 것과 같은 불완전급부 등으로 인하여 발생한 손해는 그것이 그 부실공사 등과 상당인과관계가 있는 완공의 지체로

### 5. LXML로 본문 저장하기

In [5]:
!pip install lxml

In [46]:
import lxml
from lxml import etree as et
from io import StringIO, BytesIO

In [40]:
# 특정 판례일련번호의 lxml element를 가져오는 함수입니다.
def myfunction(case_number):
    main = {'OC':'ICTPoolC',
           'target':'prec',
           'ID': case_number,
          'type':'XML'}
    res_main = rq.get('http://www.law.go.kr/DRF/lawService.do?',params=main)
    res_main = re.sub(' encoding="UTF-8"', '', res_main.text) 
    return et.fromstring(res_main)

In [41]:
#  특정 판례 lxml element에서 원하는 tag의 내용을 가져오는 함수입니다.
def findtext(tree, tag):
    for el in tree.iter():
        if (el.tag == tag):
            return re.sub('<.*?>','', el.text)

In [48]:
def save_local(list_xml, output_path):
    
    data = re.sub(' encoding="UTF-8"', '', list_xml.text)
    data = et.XML(data)
    
    for el in data.iter():
        if (el.tag == "판례일련번호"):
            f = open(os.getcwd() + '\\' + output_path + '\\' + el.text + ".txt", 'w')
            f.write(findtext(myfunction(el.text), '판례정보일련번호') + '\n')
            f.write(findtext(myfunction(el.text), '사건명') + '\n')
            f.write(findtext(myfunction(el.text), '판시사항') + '\n')
            f.write(findtext(myfunction(el.text), '판결요지') + '\n')
            f.write(findtext(myfunction(el.text), '판례내용') + '\n')
            f.close()
            print(el.text + ' : saved')

In [51]:
def txt_concat(path, output_name):
    
    file_list = os.listdir(path)
    output = open(output_name + '.txt', 'w')
    for name in file_list:
        if ".txt" not in name:
            continue
        f = open(path + '\\' + name, 'r')
        for line in f:
            output.write(line)
        output.write('\n')
    
        f.close()
    output.close()

In [44]:
def all_in_one(keyword, case_num):
    ## 목록 request templat
    chart= {'OC':'ICTPoolC'
            ,'target':'prec'
            ,'type':'XML'
            ,'query': keyword
            ,'display': case_num
            ,'curt':'대법원'
            ,'prncYd':'20000101~20191231'
           }
    
    ## 목록 받아오기
    res_chart=rq.get('http://www.law.go.kr/DRF/lawSearch.do?',params=chart)
    
    save_local(res_chart, 'Case_Main')
    txt_concat(os.getcwd() + '\\Case_Main\\', 'total_case')

In [49]:
all_in_one('손해배상', 3000)

210813 : saved
210827 : saved
210829 : saved
210569 : saved
210415 : saved
210423 : saved
210419 : saved
210853 : saved
210895 : saved
209687 : saved
209693 : saved
209587 : saved
209131 : saved
208800 : saved
208795 : saved
208726 : saved
208733 : saved
208734 : saved
208744 : saved
208745 : saved
208415 : saved
207529 : saved
207541 : saved
207485 : saved
207486 : saved
207490 : saved
207140 : saved
207095 : saved
207100 : saved
207107 : saved
206286 : saved
205697 : saved
205700 : saved
205791 : saved
205368 : saved
204520 : saved
204471 : saved
204473 : saved
204489 : saved
204501 : saved
204506 : saved
208848 : saved
204343 : saved
208849 : saved
204331 : saved
204335 : saved
204347 : saved
207581 : saved
204219 : saved
207580 : saved
207579 : saved
204226 : saved
204230 : saved
207594 : saved
207588 : saved
204247 : saved
209581 : saved
206325 : saved
206320 : saved
206327 : saved
204270 : saved
206331 : saved
206330 : saved
204274 : saved
204295 : saved
204297 : saved
206338 : s

168610 : saved
190361 : saved
170997 : saved
168432 : saved
168093 : saved
171031 : saved
170775 : saved
170806 : saved
167834 : saved
167838 : saved
167843 : saved
170810 : saved
170798 : saved
170700 : saved
170699 : saved
170400 : saved
170370 : saved
166987 : saved
167192 : saved
170369 : saved
166730 : saved
166751 : saved
169580 : saved
166753 : saved
166947 : saved
166949 : saved
167191 : saved
168232 : saved
169231 : saved
166511 : saved
168941 : saved
166369 : saved
166360 : saved
166370 : saved
165922 : saved
168337 : saved
168343 : saved
168334 : saved
168345 : saved
168368 : saved
167443 : saved
167493 : saved
165433 : saved
167545 : saved
165179 : saved
165210 : saved
167507 : saved
167328 : saved
164657 : saved
167310 : saved
167361 : saved
164521 : saved
167302 : saved
167297 : saved
167272 : saved
167290 : saved
167273 : saved
167276 : saved
167271 : saved
164196 : saved
164198 : saved
167291 : saved
167231 : saved
167237 : saved
167251 : saved
192395 : saved
163904 : s

69206 : saved
85533 : saved
85495 : saved
85496 : saved
85493 : saved
85492 : saved
69186 : saved
145345 : saved
69160 : saved
85464 : saved
126085 : saved
69163 : saved
85469 : saved
69142 : saved
85444 : saved
85441 : saved
85446 : saved
69138 : saved
85418 : saved
85419 : saved
69106 : saved
85416 : saved
69067 : saved
69066 : saved
69070 : saved
85389 : saved
69065 : saved
85384 : saved
85383 : saved
85406 : saved
69063 : saved
69060 : saved
85382 : saved
69064 : saved
69031 : saved
126076 : saved
85356 : saved
126155 : saved
69016 : saved
69014 : saved
69015 : saved
85331 : saved
85332 : saved
85336 : saved
68991 : saved
193637 : saved
68971 : saved
85313 : saved
85316 : saved
68967 : saved
85314 : saved
85305 : saved
85308 : saved
85310 : saved
85306 : saved
68932 : saved
145336 : saved
68918 : saved
85279 : saved
85278 : saved
68920 : saved
68897 : saved
68906 : saved
68889 : saved
149032 : saved
68896 : saved
85255 : saved
68860 : saved
85223 : saved
68856 : saved
68861 : saved

81235 : saved
81224 : saved
81223 : saved
81194 : saved
81196 : saved
81153 : saved
191835 : saved
191839 : saved
191829 : saved
191825 : saved
81117 : saved
191797 : saved
81074 : saved
191757 : saved
81073 : saved
191725 : saved
191727 : saved
191719 : saved
191721 : saved
192638 : saved
203745 : saved
192650 : saved
80983 : saved
192667 : saved
191675 : saved
192580 : saved
192585 : saved
80943 : saved
80935 : saved
192602 : saved
192610 : saved
191654 : saved
191652 : saved
191648 : saved
191650 : saved
191647 : saved
80870 : saved
191617 : saved
191618 : saved
191612 : saved
80850 : saved
191581 : saved
191587 : saved
191556 : saved
80774 : saved
80766 : saved
191539 : saved
191531 : saved
191530 : saved
191526 : saved
80719 : saved
191500 : saved
80689 : saved
191507 : saved
191496 : saved
191505 : saved
191490 : saved
191488 : saved
80672 : saved
191467 : saved
191468 : saved
191465 : saved
80640 : saved
80593 : saved
191429 : saved
191426 : saved
191425 : saved
191424 : saved
1

FileNotFoundError: [Errno 2] No such file or directory: '124844.txt'

In [52]:
txt_concat(os.getcwd() + '\\Case_Main\\', 'total_case')